In [2]:
import requests
import urllib
import urllib.request as req
from bs4 import BeautifulSoup as bs
import selenium
from selenium import webdriver
from IPython.display import display
import pandas as pd
import numpy as np
import os
import datetime
import time
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from apiclient.discovery import build
import random
import json
import pickle

In [3]:
#오늘 날씨 reminder
#네이버 크롤링
html = requests.get('https://search.naver.com/search.naver?where=nexearch&sm=top_hty&fbm=1&ie=utf8&query=%EB%82%A0%EC%94%A8')
soup = bs(html.text,'html.parser')
Wdata = soup.find('div', {'class':'weather_box'})
find_lc = Wdata.find('span',{'class':'btn_select'}).text #현위치
find_ct = Wdata.find('span',{'class':'todaytemp'}).text  #현재온도
find_ht = Wdata.select_one('span.max>span.num').text #최고온도
find_lt = Wdata.select_one('span.min>span.num').text  #최저온도
find_ft = Wdata.select_one('span.sensible>em>span.num').text#체감온도
find_dt = Wdata.select_one('div.today_area._mainTabContent > div.main_info > div > ul > li:nth-child(1) > p').text  #차이
find_am_hm = Wdata.select_one('li>span.point_time.morning>span.rain_rate>span.num').text #오전강우확률
find_pm_hm = Wdata.select_one('li>span.point_time.afternoon>span.rain_rate>span.num').text #오후강우확률

#미세먼지
html = requests.get('https://search.naver.com/search.naver?where=nexearch&sm=top_hty&fbm=1&ie=utf8&query=%EB%82%A0%EC%94%A8')
soupd = bs(html.text,'html.parser')
Ddata = soupd.find('div', {'class':'weather_box'})
find_du = Ddata.select_one('dl.indicator>dd>span.num').text

ev_du = int(find_du.strip('㎍/㎥'))
if ev_du <= 30:
    ev_dust = "좋음"
elif 30 <ev_du<= 80:
    ev_dust = "보통"
elif 80 <ev_du<= 150:
    ev_dust = "나쁨"
else:
    ev_dust = "매우나쁨"

find_M_du = Ddata.select('dl.indicator>dd>span.num')[1].text
ev_M_du = find_M_du.strip('㎍/㎥')

try: 
    ev_M_du = int(ev_M_du)
    if ev_M_du <= 15:
        ev_M_dust = "좋음"
    elif 15 <ev_M_du<= 50:
        ev_M_dust = "보통"
    elif 51 <ev_M_du<= 100:
        ev_M_dust = "나쁨"
    else:
        ev_M_dust = "매우나쁨"
except ValueError as exception:
    ev_M_dust = "너무좋음"
    
t_dust = str(ev_du)
u_dust = str(ev_M_du)

av_rain = (int(find_am_hm) + int(find_pm_hm))/2
av_rain = int(av_rain)
av_rain # 비오는날 측정 평균값. 70% 이상 비온다고 판단. 

c_state = find_dt.split(',')[0] #현재 기상상황
d_temp = find_dt.split(',')[1].strip() #어제와다름

In [4]:
options = webdriver.ChromeOptions()
options.headless = True
options.add_argument('window-size-1920x1080')
url = 'https://weather.naver.com/'
driver = webdriver.Chrome('D:/Tools/chromedriver.exe',options=options)
driver.get(url)

In [5]:
uv_st = driver.find_element_by_xpath('//*[@id="lifeIndex"]/div[1]/ul/li[1]/div[1]/strong').text
uv_st

'보통'

In [6]:
weather = {}
weather['tmp'] = find_ct +"°"  #현재기온
weather['h_tmp'] = find_ht +"°" #오늘 최고기온
weather['L_tmp'] = find_lt +"°" #오늘 최저기온
weather['f_tmp'] = find_ft +"°"# 현재 체감온도
weather['c_sta'] = c_state # 어제와 다른
weather['d_temp'] = d_temp #어제와 다른 온도
weather['m_rain'] = find_am_hm + "%" # 오전 강우확률
weather['d_rain'] = find_pm_hm + "%" # 오후 강우확률
weather['t_dust'] = t_dust + "㎍/㎥"  #오늘 미세먼지
weather['t_dust_s'] = ev_dust
weather['u_dust'] = u_dust + "㎍/㎥"#오늘 초 미세먼지
weather['u_dust_s'] = ev_M_dust 
weather['t_uv'] = uv_st # 자외선지수
weather

{'tmp': '24°',
 'h_tmp': '33°',
 'L_tmp': '25°',
 'f_tmp': '26°',
 'c_sta': '비',
 'd_temp': '어제보다 1˚ 낮아요',
 'm_rain': '60%',
 'd_rain': '60%',
 't_dust': '2㎍/㎥',
 't_dust_s': '좋음',
 'u_dust': '2㎍/㎥',
 'u_dust_s': '좋음',
 't_uv': '보통'}

In [7]:
#오늘 일자
today = datetime.date.today().isoformat()
today

'2021-07-19'

In [8]:
#날씨정보
weather_text = f'''\
오늘의 날씨({today})
현재온도 : {weather['tmp']}
최고/최저 : {weather['h_tmp']+'/'+weather['L_tmp']}
체감온도 : {weather['f_tmp']}
현재상황 : {weather['c_sta']}
어제비교 : {weather['d_temp']}
오전 강우확률 : {weather['m_rain']}
오후 강우확률 : {weather['d_rain']}
미세먼지 : {weather['t_dust'] , weather['t_dust_s']}
초미세먼지 : {weather['u_dust'], weather['u_dust_s']}
자외선 : {weather['t_uv']}
'''

In [9]:
weather_text

"오늘의 날씨(2021-07-19)\n현재온도 : 24°\n최고/최저 : 33°/25°\n체감온도 : 26°\n현재상황 : 비\n어제비교 : 어제보다 1˚ 낮아요\n오전 강우확률 : 60%\n오후 강우확률 : 60%\n미세먼지 : ('2㎍/㎥', '좋음')\n초미세먼지 : ('2㎍/㎥', '좋음')\n자외선 : 보통\n"

In [ ]:
#구글 캘린더 api
#엑세스 요청사항
gcred_file = '../Credit.json'
scope = ['https://www.googleapis.com/auth/calendar']
flow = InstalledAppFlow.from_client_secrets_file(gcred_file, scope)
gcreds = flow.run_local_server()

In [ ]:
service = build('calendar','v3',credentials=gcreds)
calendar_id = 'o9s9ke801ob4e8lkojmqefoe8k@group.calendar.google.com' # 사용할 캘린더 id
today = datetime.date.today().isoformat()
time_min = today + 'T00:00:00+09:00'
time_max = today + 'T23:59:59+09:00'
max_results = 1 #일정 조회 최대 개수
is_single_events = True #반복일정여부
orderby = 'startTime' #일정 정렬

#오늘 일정 가져오기
events_result = service.events().list(calendarId = calendar_id,
                                     timeMin = time_min,
                                     timeMax = time_max,
                                     maxResults = max_results,
                                     singleEvents = is_single_events,
                                     orderBy = orderby).execute()


In [ ]:
items = events_result.get('items')
item = items[0]

In [ ]:
start_date = item['start']['dateTime']
s_day = start_date.split('T')[0]
s_time = start_date.split('T')[1]
s_time = s_time.split('+')[0]

end_date = item['end']['dateTime']
e_day = end_date.split('T')[0]
e_time = end_date.split('T')[1]
e_time = e_time.split('+')[0]

In [ ]:
location = item['location'].split(',')[0].strip()

In [ ]:
schedule = {}

schedule['title'] = item.get('summary')
schedule['category'] = item['organizer']['displayName']
schedule['details'] = item.get('description')
schedule['location'] = item.get('location')
schedule['s_day'] = s_day
schedule['s_time'] = s_time
schedule['e_day'] = e_day
schedule['e_time'] = e_time


In [ ]:
schedule

In [ ]:
schedule_text =f'''\
#오늘의 해야할것 : {schedule['title']}
스케쥴 카테고리 : {schedule['category']}
스케쥴 세부사항 : {schedule['details']}
스케쥴 위치 : {schedule['location']}
시작일시 : {schedule['s_day'],schedule['s_time']}
종료일시 : {schedule['e_day'],schedule['e_time']}
'''

In [ ]:
schedule_text

In [10]:
#음식추천

In [11]:
ncreds = {
    'client_id' : "IS7Lmhp7lT6z4uFNjEG5",
    "client_secret" : "05CLjBMpLC"    
}
nheaders = {
    "X-Naver-Client-Id" : ncreds.get('client_id'),
    "X-Naver-Client-Secret" : ncreds.get('client_secret')
}

In [12]:
#기온별 음식
low_temp = "부대찌개,해물탕,칼국수,국밥,마라탕,우동,청국장,된장찌개,김치찌개,국수,소주".split(',')
high_temp = "냉면,소바,비빔국수,맥주,밀면,샐러드,베이글,돈까스,카레,짜장면".split(',')
rain = '파전,빈대떡,전'.split(',')

In [13]:
if 25 <= int(find_ct):
    weather = 'high'
elif 18 > int(find_ct):
    weather = 'low'
elif 70 < av_rain:
    weather = 'rain'
else:
    weather = 'fine'
    
    

In [14]:
import random
foods = None #아무 것도 없는 변수 생성
if weather == 'high' :
    foods = random.sample(high_temp,k=len(high_temp))
elif weather == 'low' :
    foods = random.sample(low_temp,k=len(low_temp))
elif weather == 'rain':
    foods = random.sample(rain,k=len(rain))
else:
    foods=['']
    
foods

['']

In [15]:
location1= find_lc

In [16]:
params_format = "sort=comment&query="
naver_url = "https://openapi.naver.com/v1/search/local.json?"
params_format = 'sort=comment&query='
recommands = []
location = location1
for food in foods:
    query = location1 + " " + food + '맛집'
    params = params_format + query + '&display=' + '5'
    res = requests.get(naver_url+params, headers=nheaders)
    result = res.json().get('items')
    
    if weather == 'fine':
        for i in range(0,3):
            recommands.append(result[i])
        break
        
    if result:
        recommands.append(result[0])
        if len(recommands) >= 3:
            break

In [17]:
recommands

[{'title': '금포칼국수',
  'link': '',
  'category': '한식>칼국수,만두',
  'description': '',
  'telephone': '',
  'address': '경기도 김포시 장기동 2023-1',
  'roadAddress': '경기도 김포시 김포한강2로23번길 60 1층',
  'mapx': '283687',
  'mapy': '560631'},
 {'title': '빵선생 운양점',
  'link': 'http://www.bbangsseam.com/',
  'category': '카페,디저트>베이커리',
  'description': '',
  'telephone': '',
  'address': '경기도 김포시 운양동 1298-3 헤리움타운 1층',
  'roadAddress': '경기도 김포시 김포한강1로 247 헤리움타운 1층',
  'mapx': '284127',
  'mapy': '562127'},
 {'title': '쿠우쿠우 김포운양점',
  'link': 'http://www.qooqoo.co.kr/',
  'category': '음식점>일식,초밥뷔페',
  'description': '',
  'telephone': '',
  'address': '경기도 김포시 운양동 1307-3',
  'roadAddress': '경기도 김포시 김포한강1로 250 골든타임 9층',
  'mapx': '284171',
  'mapy': '562035'}]

In [18]:
kcreds = {
    "access_token" : "VuisqIwpcFVb13bVd0O4MS4qlAV2QQpP_LNEUQopcNEAAAF6uit4-w"
}
kheaders = {
    "Authorization": "Bearer " + kcreds.get('access_token')
}

In [19]:
import json

# 카카오톡 URL 주소
kakaotalk_template_url = "https://kapi.kakao.com/v2/api/talk/memo/default/send"

# 날씨 상세 정보 URL
weather_url = "https://search.naver.com/search.naver?sm=top_hty&fbm=0&ie=utf8&query=%EB%82%A0%EC%94%A8"

# 날씨 정보 만들기 
text = weather_text

# 텍스트 템플릿 형식 만들기
template = {
  "object_type": "text",
  "text": text,
  "link": {
    "web_url": weather_url,
    "mobile_web_url": weather_url
  },
  "button_title": "날씨 상세보기"
}

# JSON 형식 -> 문자열 변환
payload = {
    "template_object" : json.dumps(template)
}

# 카카오톡 보내기
res = requests.post(kakaotalk_template_url, data=payload, headers=kheaders)

if res.json().get('result_code') == 0:
    print('메시지를 성공적으로 보냈습니다.')
else:
    print('메시지를 성공적으로 보내지 못했습니다. 오류메시지 : ' + str(res.json()))

메시지를 성공적으로 보냈습니다.


In [ ]:
import json

# 카카오톡 URL 주소
kakaotalk_template_url = "https://kapi.kakao.com/v2/api/talk/memo/default/send"

# 스케쥴 상세 정보 URL
schedule_url = "https://search.naver.com/search.naver?where=nexearch&sm=top_hty&fbm=0&ie=utf8&query=%EA%B5%AC%EA%B8%80%EC%BA%98%EB%A6%B0%EB%8D%94"

# 스케쥴 정보 만들기 
text = schedule_text

# 텍스트 템플릿 형식 만들기
template = {
  "object_type": "text",
  "text": text,
  "link": {
    "web_url": schedule_url,
    "mobile_web_url": schedule_url
  },
  "button_title": "스케쥴 상세보기"
}

# JSON 형식 -> 문자열 변환
payload = {
    "template_object" : json.dumps(template)
}

# 카카오톡 보내기
res = requests.post(kakaotalk_template_url, data=payload, headers=kheaders)

if res.json().get('result_code') == 0:
    print('메시지를 성공적으로 보냈습니다.')
else:
    print('메시지를 성공적으로 보내지 못했습니다. 오류메시지 : ' + str(res.json()))

In [20]:
contents = []
template = {
    "object_type" : "list",
    "header_title" : "현재 날씨에 따른 음식 추천",
    "header_link" : {
        "web_url": weather_url,
        "mobile_web_url" : weather_url
    },
    "contents" : contents,
    "buttons" : [
        {
            "title" : "날씨 정보 상세보기",
            "link" : {
                "web_url": weather_url,
                "mobile_web_url" : weather_url
            }
        }
    ],
}

# contents 만들기
for place in recommands:
    title = place.get('title')  # 장소 이름
    # title : 태극쿵푸<b>마라탕</b>
    # html 태그 제거
    title = title.replace('<b>','').replace('</b>','')
    
    category = place.get('category')  # 장소 카테고리
    telephone = place.get('telephone')  # 장소 전화번호
    address = place.get('address')  # 장소 지번 주소

    # 각 장소를 클릭할 때 네이버 검색으로 연결해주기 위해 작성된 코드
    enc_address = urllib.parse.quote(address + ' ' + title)
    query = "query=" + enc_address

    # 장소 카테고리가 카페이면 카페 이미지
    # 이외에는 음식 이미지
    if '카페' in category:
        image_url = "https://freesvg.org/img/pitr_Coffee_cup_icon.png"
    else:
        image_url = "https://freesvg.org/img/bentolunch.png?w=150&h=150&fit=fill"

    # 전화번호가 있다면 제목과 함께 넣어줍니다.
    if telephone:
        title = title + "\ntel) " + telephone

    # 카카오톡 리스트 템플릿 형식에 맞춰줍니다.
    content = {
        "title": "[" + category + "] " + title,
        "description": ' '.join(address.split()[1:]),
        "image_url": image_url,
        "image_width": 50, "image_height": 50,
        "link": {
            "web_url": "https://search.naver.com/search.naver?" + query,
            "mobile_web_url": "https://search.naver.com/search.naver?" + query
        }
    }
    
    contents.append(content)

# JSON 형식 -> 문자열 변환
payload = {
    "template_object" : json.dumps(template)
}

# 카카오톡 보내기
res = requests.post(kakaotalk_template_url, data=payload, headers=kheaders)

if res.json().get('result_code') == 0:
    print('메시지를 성공적으로 보냈습니다.')
else:
    print('메시지를 성공적으로 보내지 못했습니다. 오류메시지 : ' + str(res.json()))

메시지를 성공적으로 보냈습니다.


In [ ]:
gejofuYWaKXSlVs6mMsplq9urTlBOW8h0Vean3p2kESik7QO2H72zPLN155v7SGHqNXntwopdSkAAAF6uYXwwg

In [35]:
import requests
import json

url = "https://kauth.kakao.com/oauth/token"

data = {
    "grant_type" : "authorization_code",
    "client_id" : "6f6887a7f4c56636b244ffbe9c9017dc",
    "redirect_uri" : "https://localhost.com",
    "code"         : "gejofuYWaKXSlVs6mMsplq9urTlBOW8h0Vean3p2kESik7QO2H72zPLN155v7SGHqNXntwopdSkAAAF6uYXwwg"
    
}
response = requests.post(url, data=data)

tokens = response.json()

print(tokens)


{'access_token': 'ZDsbSPtkDlObJX4r89G_rqPG_C7L0lSs69LBLQopyV8AAAF6uYgB2A', 'token_type': 'bearer', 'refresh_token': 'Qakw1dLIKtDHUhJ5HFx01pWPSJ0NlmtoiHH1cwopyV8AAAF6uYgB1g', 'expires_in': 21599, 'scope': 'account_email profile_image talk_message profile_nickname', 'refresh_token_expires_in': 5183999}
